In [50]:
import pandas as pd
import pipes as pp
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score

# 1.We review the information via transformer. Preparing data for processing format

In [51]:
df = pd.read_csv('train.csv')

df = df.drop(['CustomerID','ID','Name','TypeofLoan','SSN'], axis = 1)

In [52]:
df = pp.myPipe.transform(df)

In [53]:
df

,Month,Age,AnnualIncome,MonthlyInhandSalary,NumBankAccounts,NumCreditCard,InterestRate,NumofLoan,Delayfromduedate,NumofDelayedPayment,...,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,1,2,3,4,5,6,7
0,0.0,23.0,19114.12,1824.843333,3,4,3,4.0,3,7.0,...,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,23.0,19114.12,0.000000,3,4,3,4.0,-1,0.0,...,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,19114.12,0.000000,3,4,3,4.0,3,7.0,...,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3.0,23.0,19114.12,0.000000,3,4,3,4.0,5,4.0,...,1,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4.0,23.0,19114.12,1824.843333,3,4,3,4.0,6,0.0,...,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3.0,25.0,39628.99,3359.415833,4,6,7,2.0,23,7.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
99996,4.0,25.0,39628.99,3359.415833,4,6,7,2.0,18,7.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
99997,5.0,25.0,39628.99,3359.415833,4,6,5729,2.0,27,6.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
99998,6.0,25.0,39628.99,3359.415833,4,6,7,2.0,20,0.0,...,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [54]:
X = df.drop(['CreditScore','PaymentBehaviour'], axis = 1)

In [55]:
Y = df['CreditScore']

# Создание проверочного набора (Create a validation set)

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size = 0.5, random_state = 0)

In [57]:
X_train.shape, Y_train.shape

((50000, 43), (50000,))

# a. ML With Teacher

# Отбор признаков (feature selection)

In [32]:
#WE SHOULD DO IT BEFORE MINMAX SCALER!

In [58]:
from sklearn.feature_selection import SelectKBest,chi2,f_classif
#специальные методики,оценивают, какие признаки хорошо влияют на зависимые переменные,какие плохо и т.д.(хи квадрат используем)
#нужно передать количество признаков,которое мы хотим отобрать и критерий по которому будем отбирать(наш хи-квадрат)

Selection using the chi function

In [59]:
#k - количество признаков
Selector = SelectKBest(f_classif, k=9)
Selector.fit(X, Y)

SelectKBest(k=9)

In [60]:
#покажет признаки,которые он выбирает,что подходит нам и что нет,проверит степень влияния
Selector.get_support()


array([False, False, False,  True, False, False, False, False,  True,
       False,  True, False,  True,  True, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True])

In [61]:
X.columns[Selector.get_support()].tolist()

['MonthlyInhandSalary',
 'Delayfromduedate',
 'ChangedCreditLimit',
 'CreditMix',
 'OutstandingDebt',
 'PaymentofMinAmount',
 'CreditHistoryYears',
 '4',
 '7']

In [62]:
drop_cols = X.columns[~Selector.get_support()].tolist()

In [63]:
drop_cols

['Month',
 'Age',
 'AnnualIncome',
 'NumBankAccounts',
 'NumCreditCard',
 'InterestRate',
 'NumofLoan',
 'NumofDelayedPayment',
 'NumCreditInquiries',
 'CreditUtilizationRatio',
 'TotalEMIpermonth',
 'Amountinvestedmonthly',
 'MonthlyBalance',
 'Occupation_0',
 'Occupation_Accountant',
 'Occupation_Architect',
 'Occupation_Developer',
 'Occupation_Doctor',
 'Occupation_Engineer',
 'Occupation_Entrepreneur',
 'Occupation_Journalist',
 'Occupation_Lawyer',
 'Occupation_Manager',
 'Occupation_Mechanic',
 'Occupation_MediaManager',
 'Occupation_Musician',
 'Occupation_Scientist',
 'Occupation_Teacher',
 'Occupation_Writer',
 '1',
 '2',
 '3',
 '5',
 '6']

In [67]:
X_new = X.drop(drop_cols, axis = 1)

In [68]:
X_new

,MonthlyInhandSalary,Delayfromduedate,ChangedCreditLimit,CreditMix,OutstandingDebt,PaymentofMinAmount,CreditHistoryYears,4,7
0,1824.843333,3,11.27,0.0,809.98,1,22.0,0.0,0.0
1,0.000000,-1,11.27,1.0,809.98,1,0.0,0.0,0.0
2,0.000000,3,0.00,1.0,809.98,1,22.0,0.0,0.0
3,0.000000,5,6.27,1.0,809.98,1,22.0,1.0,0.0
4,1824.843333,6,11.27,1.0,809.98,1,22.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
99995,3359.415833,23,11.50,0.0,502.38,1,31.0,0.0,1.0
99996,3359.415833,18,11.50,0.0,502.38,1,31.0,0.0,0.0
99997,3359.415833,27,11.50,1.0,502.38,1,31.0,0.0,1.0
99998,3359.415833,20,11.50,1.0,502.38,1,31.0,0.0,0.0


In [71]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,  Y_test = train_test_split(X_new, Y, test_size = 0.5, random_state = 0)

In [72]:
X_train.shape

(50000, 9)

# scaling and standardization of values

Standardize features by removing the mean and scaling to unit variance.
Standardization of a dataset is a common requirement for many machine learning estimators: 
they might behave badly if the individual features do not more or less look like standard normally distributed data.

Method fit_transform:Fits transformer to X and y with optional parameters fit_params and returns a transformed version of X.

In [73]:
#from sklearn.preprocessing import StandardScaler
#mms = StandardScaler()
#X = mms.fit_transform(X)

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X = mms.fit_transform(X_train)

# Адаптивный бустинг (Adaptive boosting)

In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
#clf = tree.DecisionTreeClassifier()
from sklearn.tree import DecisionTreeClassifier  ### Замена класса
from sklearn.ensemble import RandomForestClassifier

In [75]:
Ada_clf = AdaBoostClassifier()

In [76]:
#Ada_clf = AdaBoostClassifier(base_estimator = tree.DecisionTreeClassifier(max_depth = 7))
paramgrid = [{'base_estimator': [DecisionTreeClassifier(max_depth = 5)], 
              'learning_rate' : [0.1,0.5,1], 'n_estimators': [10,100,200]}, 
             
             
             {'base_estimator': [RandomForestClassifier(max_depth = 2, n_estimators = 10)], 
              'learning_rate' : [0.1,1],
              'n_estimators': [10]}
            ]

In [77]:
GS_ada = GridSearchCV(Ada_clf, param_grid = paramgrid, cv = 3, scoring = 'f1')

In [78]:
GS_ada.fit(X, Y_train)

GridSearchCV(cv=3, estimator=AdaBoostClassifier(),
             param_grid=[{'base_estimator': [DecisionTreeClassifier(max_depth=5)],
                          'learning_rate': [0.1, 0.5, 1],
                          'n_estimators': [10, 100, 200]},
                         {'base_estimator': [RandomForestClassifier(max_depth=2,
                                                                    n_estimators=10)],
                          'learning_rate': [0.1, 1], 'n_estimators': [10]}],
             scoring='f1')

In [79]:
GS_ada.best_params_

{'base_estimator': DecisionTreeClassifier(max_depth=5),
 'learning_rate': 0.5,
 'n_estimators': 200}

In [80]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),learning_rate = 0.5, n_estimators = 200)
ada.fit(X,Y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=200)

In [82]:
from sklearn.metrics import f1_score
pred = ada.predict(X_test)
f1_score = (Y_test,pred)

In [84]:
from sklearn.model_selection import cross_val_score, cross_val_predict

In [86]:
cs = cross_val_score(GS_ada,X, Y_train, cv= 5, scoring = 'f1')

KeyboardInterrupt: 

In [ ]:
cs

In [37]:
ada.score(X_test,Y_test), Ada_clf.score(X_train, Y_train)

(0.6768, 0.765575)

In [157]:
pred = ada.predict(X_test)

In [158]:
pred

array([0, 1, 1, ..., 1, 2, 0], dtype=int64)

In [159]:
confusion_matrix(Y_test,pred)


array([[5517, 1188,  725],
       [1545, 2417,  131],
       [ 981,   71, 1409]], dtype=int64)

In [160]:
precision_score(Y_test,pred,pos_label='positive', average='micro'), recall_score(Y_test,pred,pos_label='positive', average='micro'), f1_score(Y_test,pred,pos_label='positive', average='micro') 

C:\Users\laptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
C:\Users\laptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
C:\Users\laptop\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


(0.6681207093821511, 0.6681207093821511, 0.6681207093821511)